In [16]:
import torch
#!pip install torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='./data', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

print(data)
print("Nodes:", data.num_nodes)
print("Edges:", data.edge_index.shape[1])
print("Features per node:", dataset.num_features)
print("Num classes:", dataset.num_classes)
print(data.x.shape)
print(data.y.shape)
print ("Train/Val/Test sizes:", int(data.train_mask.sum()), int(data.val_mask.sum()), int(data.test_mask.sum()))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
      super(GAT, self).__init__()
      self.dropout = nn.Dropout(dropout)
      self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
      self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, x, edge_index):
      x = self.dropout(x)
      x = self.conv1(x, edge_index)
      x = F.elu(x)
      x = self.dropout(x)
      x = self.conv2(x, edge_index)
      return x

# Test to make sure out.shape matches data.num_nodes, and dataset.num_classes
#model = GAT(in_channels=dataset.num_features,
#            hidden_channels=8,
#            out_channels=dataset.num_classes,
#            heads=8,
#            dropout=0.6)
#out = model(data.x, data.edge_index)
#print("Expected Shape:", out.shape)  # should be [data.num_nodes, dataset.num_classes]




Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Nodes: 2708
Edges: 10556
Features per node: 1433
Num classes: 7
torch.Size([2708, 1433])
torch.Size([2708])
Train/Val/Test sizes: 140 500 1000
Expected Shape: torch.Size([2708, 7])
